In [94]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import logging
from datetime import datetime
from transformers import AutoTokenizer, AutoModel
import torch


# Ensure nltk stopwords are downloaded
nltk.download('stopwords')

# Set up logging
logging.basicConfig(level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abbasm1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
def preprocess_text(text):
    if pd.isnull(text):
        return ''
    # Lowercase the text
    text = text.lower()
    # Remove numbers and special characters
    text = re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    # Remove stopwords
    stop_words = set(stopwords.words('german'))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back to string
    text = ' '.join(tokens)
    return text

def extract_state_city(location):
    if pd.isnull(location):
        return ''
    parts = location.split('>')
    parts = [part.strip() for part in parts]
    if len(parts) >= 2:
        return ' > '.join(parts[:2])
    else:
        return location.strip()
    
def _extract_location_parts(location) :
        """Extract and categorize location parts into states and cities."""
        locations = {
            'states': set(),
            'cities': set()
        }
        german_states = {
            'baden-württemberg', 'bayern', 'berlin', 'brandenburg', 'bremen',
            'hamburg', 'hessen', 'mecklenburg-vorpommern', 'niedersachsen',
            'nordrhein-westfalen', 'rheinland-pfalz', 'saarland', 'sachsen',
            'sachsen-anhalt', 'schleswig-holstein', 'thüringen'
        }

        if pd.isna(location):
            return locations
        
        try:
            # Split on common delimiters and clean up
            parts = re.split(r'[>/,\n]\s*', str(location))
            for part in parts:
                part = part.strip().lower()
                if part:
                    # Check if it's a state
                    if part in german_states:
                        locations['states'].add(part)
                    else:
                        # Remove "region" and other common prefixes
                        clean_part = re.sub(r'^region\s+', '', part)
                        if clean_part:
                            locations['cities'].add(clean_part)
                    
        except Exception as e:
            logging.error(f"Error extracting location parts: {e}")
        
        state_city = ", ".join(locations['states'] | locations['cities'])

        return state_city

def combine_text_fields(row):
    return row['title_preprocessed'] + ' ' + row['description_preprocessed'] + ' ' + row['long_description_preprocessed'] 




In [96]:
# Load buyer and seller datasets
buyers_df = pd.read_csv('./data/nexxt_change_purchase_listings.csv')
# buyers_df = pd.read_csv('./data/nexxt_change_purchase_listings.csv')
sellers_df = pd.read_csv('./data/nexxt_change_sales_listings_updates.csv')

In [97]:
# Preprocess buyers' text fields
logging.info('Preprocessing buyers\' text fields...')
buyers_df['title_preprocessed'] = buyers_df['title'].apply(preprocess_text)
buyers_df['description_preprocessed'] = buyers_df['description'].apply(preprocess_text)
buyers_df['long_description_preprocessed'] = buyers_df['long_description'].apply(preprocess_text)
# buyers_df['branchen_preprocessed']= buyers_df.apply(
#     lambda row: " ".join(filter(pd.notna, [row.get('Industrie'), row.get('Sub-Industrie')])),
#     axis=1
# )
        

# Preprocess sellers' text fields
logging.info('Preprocessing sellers\' text fields...')
sellers_df['title_preprocessed'] = sellers_df['title'].apply(preprocess_text)
sellers_df['description_preprocessed'] = sellers_df['description'].apply(preprocess_text)
sellers_df['long_description_preprocessed'] = sellers_df['long_description'].apply(preprocess_text)
# sellers_df['branchen_preprocessed'] = sellers_df.apply(
#     lambda row: " ".join(filter(None, str(row.get('branchen')).split('>'))).lower() if pd.notna(row.get('branchen')) else "",
#     axis=1
# )

# Combine text fields
logging.info('Combining text fields...')
buyers_df['combined_text'] = buyers_df.apply(combine_text_fields, axis=1)
sellers_df['combined_text'] = sellers_df.apply(combine_text_fields, axis=1)
# Extract state and city from location
logging.info('Extracting state and city from location...')
buyers_df['state_city'] = buyers_df['location'].apply(_extract_location_parts)
sellers_df['state_city'] = sellers_df['location'].apply(_extract_location_parts)

INFO:root:Preprocessing buyers' text fields...
INFO:root:Preprocessing sellers' text fields...
INFO:root:Combining text fields...
INFO:root:Extracting state and city from location...


In [98]:
# Initialize the model
logging.info('Loading the Sentence Transformer model...')
# model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
# Load a pre-trained German BERT model
# model_name = 'bert-base-german-dbmdz-uncased'
# model_name = 'distiluse-base-multilingual-cased'
model_name = 'paraphrase-multilingual-mpnet-base-v2' 
model = SentenceTransformer(model_name)



# '''NOT GOOD '''
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

INFO:root:Loading the Sentence Transformer model...
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2


In [99]:
# # Function to generate sentence embeddings
# def get_embeddings(text):
#     tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**tokens)
#     return outputs.last_hidden_state.mean(dim=1)

# # # Compute semantic similarity
# # def get_compute_similarity(row):
# #     buyer_text = row['combined_text']
# #     seller_text = row['combined_text']
    
# #     buyer_embedding = get_embeddings(buyer_text)
# #     seller_embedding = get_embeddings(seller_text)
    
# #     similarity = cosine_similarity(buyer_embedding, seller_embedding)[0][0]
# #     return similarity

In [ ]:
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

import re

# Function to parse multi-line hierarchical locations using regex
def parse_location_multiline(location):
    try:
        # Match components separated by slashes, ignoring line breaks and whitespace
        matches = re.findall(r'[^/]+', location.replace('\n', ' '))
        # Clean and standardize components
        components = [match.strip() for match in matches if match.strip()]
        return components  # Return components as a list
    except Exception as e:
        print(f"Error parsing location with regex: {location}. Error: {e}")
        return []

# Function to get latitude and longitude by trying multiple levels
def get_lat_lon(location):
    geolocator = Nominatim(user_agent="geo_matcher")
    try:
        # Parse the location into levels using regex
        location_parts = parse_location_multiline(location)
        
        # Try geocoding from most specific to least specific
        for i in range(len(location_parts), 0, -1):
            try_location = " / ".join(location_parts[:i])  # Combine levels
            loc = geolocator.geocode(try_location)
            if loc:
                return loc.latitude, loc.longitude
        
        # Fallback: Try geocoding the entire location string
        loc = geolocator.geocode(location)
        if loc:
            return loc.latitude, loc.longitude
    except Exception as e:
        print(f"Geocoding error for {location}: {e}")
    return None, None

# Function to compute geodesic distance between two coordinates
def within_radius(coord1, coord2, radius_km=50):
    try:
        if None in coord1 or None in coord2:
            return False  # If any coordinate is missing, consider it not within radius
        return geodesic(coord1, coord2).km <= radius_km
    except Exception as e:
        print(f"Error computing geodesic distance: {e}")
        return False

# Updated determine_verdict function with geocoding
def determine_verdict_with_geocoding(buyer_location,seller_location, radius_km=50):
    try:
        # Get similarity score
        
        # Get geocoded locations
        buyer_coords = get_lat_lon(buyer_location)
        seller_coords = get_lat_lon(seller_location)
        
        # Check if buyer and seller are within the radius
        location_match = within_radius(buyer_coords, seller_coords, radius_km)
        
        # Determine verdict
        if location_match:
            print(f"Error determining verdict with geocoding for row: {buyer_location} >=> {seller_location}. Error: {e}")
            return "Valid Match"
        
        else:
            return "Invalid Match"
    except Exception as e:
        print(f"Error determining verdict with geocoding for row: {buyer_location}>=> {seller_location}. Error: {e}")
        return "Error"


In [ ]:
# determine_verdict_with_geocoding(buyers_df['location'],sellers_df['location'])

buyers_df['parsed_location'] = buyers_df['location'].apply(parse_location_multiline)
sellers_df['parsed_location'] = sellers_df['location'].apply(parse_location_multiline)
df['coords'] = df['location'].apply(get_lat_lon)

Error parsing location with regex: 0      Hessen / Gießen / Lahn-Dill-Kreis Hessen / Gie...
1                                                Sachsen
2                                                 Hessen
3      Nordrhein-Westfalen / Köln / Düren Nordrhein-W...
4                                      Sachsen / Leipzig
                             ...                        
755                          Sachsen / Dresden / Dresden
756                                  Nordrhein-Westfalen
757                                               Berlin
758                                               Bayern
759                          Sachsen / Dresden / Bautzen
Name: location, Length: 760, dtype: object. Error: expected string or bytes-like object, got 'Series'


Error parsing location with regex: 0             Bayern / Mittelfranken / Ansbach, Landkreis
1                 Schleswig-Holstein / Schleswig-Holstein
2              Baden-Württemberg / Karlsruhe / Heidelberg
3                                Berlin / Berlin / Berlin
4                                     Nordrhein-Westfalen
                              ...                        
4259          Niedersachsen / Braunschweig / Wolfenbüttel
4260             Baden-Württemberg / Karlsruhe / Mannheim
4261                                              Sachsen
4262                Niedersachsen / Weser-Ems / Friesland
4263    Nordrhein-Westfalen / Arnsberg / Märkischer Kreis
Name: location, Length: 4264, dtype: object. Error: expected string or bytes-like object, got 'Series'


'Invalid Match'

In [100]:
# Set similarity threshold
similarity_threshold = 0.75

In [101]:
# Initialize matches list
matches = []

# Get unique state_city values
state_city_list = buyers_df['state_city'].unique()

for state_city in state_city_list:
    # logging.info(f'Processing location: {state_city}')
    # Get buyers and sellers with this state_city
    buyers_subset = buyers_df[buyers_df['state_city'] == state_city].reset_index(drop=True)
    sellers_subset = sellers_df[sellers_df['state_city'] == state_city].reset_index(drop=True)
    
    if buyers_subset.empty or sellers_subset.empty:
        # logging.info('No matching buyers or sellers for this location.')
        continue  # No matching sellers or buyers for this location
    
    # Encode sellers' combined_text once for this state_city
    # logging.info('Encoding sellers\' text...')
    sellers_embeddings_subset = model.encode(sellers_subset['combined_text'].tolist(), convert_to_tensor=True)
    # sellers_embeddings_subset = get_embeddings(sellers_subset['combined_text'].tolist())
    
    for i, buyer_row in buyers_subset.iterrows():
        # logging.info(f'Processing buyer {i+1}/{len(buyers_subset)} in location {state_city}')
        buyer_embedding = model.encode(buyer_row['combined_text'], convert_to_tensor=True)
        # buyer_embedding = get_embeddings(buyer_row['combined_text'])
        
        # Compute cosine similarity between buyer and all sellers in this state_city
        similarity_scores = cosine_similarity(buyer_embedding.cpu().numpy().reshape(1, -1), sellers_embeddings_subset.cpu().numpy())[0]
        
        # Find indices where similarity is above threshold
        matching_sellers_indices = np.where(similarity_scores >= similarity_threshold)[0]
        
        for idx in matching_sellers_indices:
            seller_row = sellers_subset.iloc[idx]
            similarity = similarity_scores[idx]
            # Store the match
            match = {
                'buyer_title': buyer_row['title'],
                'buyer_summary': buyer_row['description'],
                'buyer_long_description': buyer_row['long_description'],
                'buyer_location': buyer_row['state_city'],
                'seller_title': seller_row['title'],
                'seller_summary': seller_row['description'],
                'seller_long_description': seller_row['long_description'],
                'seller_location': seller_row['state_city'],
                'buyer_url': buyer_row['url'],
                'seller_url': seller_row['url'],
                # 'similarity': similarity  # Optional
            }
            matches.append(match)

# Create DataFrame from matches
matches_df = pd.DataFrame(matches)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [102]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f'./matches/nlp_business_matches_{timestamp}.csv'
matches_df.to_csv(output_path, index=False)
logging.info(f'Done=> length:: {len(matches_df)} filename=> {output_path} ')

INFO:root:Done=> length:: 441 filename=> ./matches/nlp_business_matches_20241118_142054.csv 


In [115]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch

# Load buyer and seller data into a DataFrame (replace matches_df with actual data or file loading)
# Example: df = pd.read_csv('./ma')
validation_df = pd.read_csv('./data/matched_buyers_sellers.csv')
# validation_df = pd.DataFrame(matches_df)

# List of models to use
model_names = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "sentence-transformers/distiluse-base-multilingual-cased-v1"
]

# Dictionary to hold tokenizers and models
models = {}

# Load all models and tokenizers
for model_name in model_names:
    print(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    models[model_name] = {'tokenizer': tokenizer, 'model': model}

# Function to generate sentence embeddings
def get_embeddings(text, tokenizer, model):
    try:
        # Ensure text is a string and handle any NaN or None values
        if not isinstance(text, str):
            text = ""
        tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**tokens)
        # Mean Pooling
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings
    except Exception as e:
        print(f"Error generating embeddings for text: {text}. Error: {e}")
        # Return a zero tensor for failed embedding generation
        return torch.zeros((1, model.config.hidden_size))

# Function to compute semantic similarity for a given model
def compute_similarity(row, model_name):
    try:
        tokenizer = models[model_name]['tokenizer']
        model = models[model_name]['model']
        
        # Combine text fields to create buyer and seller context
        buyer_text = (
            (str(row.get('buyer_summary') or "")) + " " +
            (str(row.get('buyer_title') or "")) + " " +
            (str(row.get('buyer_long_description')) or "")
        )
        seller_text = (
            (str(row.get('seller_summary')) or "") + " " +
            (str(row.get('seller_title') or "")) + " " +
            (str(row.get('seller_long_description')) or "")
        )

        # Generate embeddings
        buyer_embedding = get_embeddings(buyer_text, tokenizer, model)
        seller_embedding = get_embeddings(seller_text, tokenizer, model)

        # Compute similarity
        similarity = cosine_similarity(buyer_embedding, seller_embedding)[0][0]
        return similarity
    except Exception as e:
        print(f"Error computing similarity for row: {row}. Error: {e}")
        return 0.0  # Return a default low similarity in case of errors

# Compute similarities for each model and store in separate columns
for model_name in model_names:
    similarity_column = f"semantic_similarity_{model_name.split('/')[-1]}"
    print(f"Computing similarities using model: {model_name}")
    validation_df[similarity_column] = validation_df.apply(lambda row: compute_similarity(row, model_name), axis=1)

# Function to determine verdict based on similarity and location for a given model
def determine_verdict(row, similarity_column, threshold=0.65):
    try:
        # Check if buyer and seller locations match
        location_match = row['buyer_location'] == row['seller_location']
        # Get the similarity score
        similarity = row[similarity_column]
        # Determine verdict
        if location_match and similarity >= threshold:
            return "Valid Match"
        else:
            return "Invalid Match"
    except Exception as e:
        print(f"Error determining verdict for row: {row}. Error: {e}")
        return "Error"

# Determine verdicts for each model and store in separate columns
for model_name in model_names:
    similarity_column = f"semantic_similarity_{model_name.split('/')[-1]}"
    verdict_column = f"verdict_{model_name.split('/')[-1]}"
    print(f"Determining verdicts using model: {model_name}")
    validation_df[verdict_column] = validation_df.apply(lambda row: determine_verdict(row, similarity_column), axis=1)

# Output the results
output_columns = [
    'buyer_title', 'seller_title', 'buyer_location', 'seller_location'
]

# Add similarity and verdict columns to the output
for model_name in model_names:
    similarity_column = f"semantic_similarity_{model_name.split('/')[-1]}"
    verdict_column = f"verdict_{model_name.split('/')[-1]}"
    output_columns.extend([similarity_column, verdict_column])

print(validation_df[output_columns])


Loading model: sentence-transformers/all-MiniLM-L6-v2
Loading model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Loading model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Loading model: sentence-transformers/distiluse-base-multilingual-cased-v1
Computing similarities using model: sentence-transformers/all-MiniLM-L6-v2
Computing similarities using model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Computing similarities using model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Computing similarities using model: sentence-transformers/distiluse-base-multilingual-cased-v1
Determining verdicts using model: sentence-transformers/all-MiniLM-L6-v2
Determining verdicts using model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Determining verdicts using model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Determining verdicts using model: sentence-transformers/distiluse-base-multilingual-cased-v1
      

In [114]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import BallTree
import logging
from datetime import datetime
from transformers import AutoTokenizer, AutoModel
import torch
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic

# Ensure nltk stopwords are downloaded
nltk.download('stopwords')

# Set up logging
logging.basicConfig(level=logging.INFO)

def preprocess_text(text):
    if pd.isnull(text):
        return ''
    # Lowercase the text
    text = text.lower()
    # Remove numbers and special characters
    text = re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    # Remove stopwords
    stop_words = set(stopwords.words('german'))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back to string
    text = ' '.join(tokens)
    return text

def extract_state_city(location):
    if pd.isnull(location):
        return ''
    parts = location.split('>')
    parts = [part.strip() for part in parts]
    if len(parts) >= 2:
        return ' '.join(parts[:2])  # Changed from ' > ' to ' ' for better geocoding
    else:
        return location.strip()

def _extract_location_parts(location):
    """Extract and categorize location parts into states and cities."""
    locations = {
        'states': set(),
        'cities': set()
    }
    german_states = {
        'baden-württemberg', 'bayern', 'berlin', 'brandenburg', 'bremen',
        'hamburg', 'hessen', 'mecklenburg-vorpommern', 'niedersachsen',
        'nordrhein-westfalen', 'rheinland-pfalz', 'saarland', 'sachsen',
        'sachsen-anhalt', 'schleswig-holstein', 'thüringen'
    }

    if pd.isna(location):
        return locations

    try:
        # Split on common delimiters and clean up
        parts = re.split(r'[>/,\n]\s*', str(location))
        for part in parts:
            part = part.strip().lower()
            if part:
                # Check if it's a state
                if part in german_states:
                    locations['states'].add(part)
                else:
                    # Remove "region" and other common prefixes
                    clean_part = re.sub(r'^region\s+', '', part)
                    if clean_part:
                        locations['cities'].add(clean_part)

    except Exception as e:
        logging.error(f"Error extracting location parts: {e}")

    state_city = ", ".join(locations['states'] | locations['cities'])

    return state_city

def combine_text_fields(row):
    return row['title_preprocessed'] + ' ' + row['description_preprocessed'] + ' ' + row['long_description_preprocessed'] 

# Geocoding setup
geolocator = Nominatim(user_agent="buyer_seller_matching")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=10.0)

def get_coordinates(location):
    if not location:
        return None, None
    try:
        loc = geocode(location + ", Germany")
        if loc:
            return loc.latitude, loc.longitude
    except Exception as e:
        logging.error(f"Geocoding error for location '{location}': {e}")
    return None, None

# Load buyer and seller datasets
logging.info('Loading buyer and seller datasets...')
buyers_df = pd.read_csv('./data/nexxt_change_purchase_listings.csv')
sellers_df = pd.read_csv('./data/nexxt_change_sales_listings_updates.csv')

# Preprocess buyers' text fields
logging.info('Preprocessing buyers\' text fields...')
buyers_df['title_preprocessed'] = buyers_df['title'].apply(preprocess_text)
buyers_df['description_preprocessed'] = buyers_df['description'].apply(preprocess_text)
buyers_df['long_description_preprocessed'] = buyers_df['long_description'].apply(preprocess_text)

# Preprocess sellers' text fields
logging.info('Preprocessing sellers\' text fields...')
sellers_df['title_preprocessed'] = sellers_df['title'].apply(preprocess_text)
sellers_df['description_preprocessed'] = sellers_df['description'].apply(preprocess_text)
sellers_df['long_description_preprocessed'] = sellers_df['long_description'].apply(preprocess_text)

# Combine text fields
logging.info('Combining text fields...')
buyers_df['combined_text'] = buyers_df.apply(combine_text_fields, axis=1)
sellers_df['combined_text'] = sellers_df.apply(combine_text_fields, axis=1)

# Extract state and city from location
logging.info('Extracting state and city from location...')
buyers_df['state_city'] = buyers_df['location'].apply(_extract_location_parts)
sellers_df['state_city'] = sellers_df['location'].apply(_extract_location_parts)

# Geocode buyers and sellers
logging.info('Geocoding buyer locations...')
buyers_df['latitude'], buyers_df['longitude'] = zip(*buyers_df['state_city'].apply(get_coordinates))

logging.info('Geocoding seller locations...')
sellers_df['latitude'], sellers_df['longitude'] = zip(*sellers_df['state_city'].apply(get_coordinates))

# Drop rows where geocoding failed
initial_buyer_count = len(buyers_df)
buyers_df = buyers_df.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)
logging.info(f"Dropped {initial_buyer_count - len(buyers_df)} buyers due to failed geocoding.")

initial_seller_count = len(sellers_df)
sellers_df = sellers_df.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)
logging.info(f"Dropped {initial_seller_count - len(sellers_df)} sellers due to failed geocoding.")

# Initialize the model
logging.info('Loading the Sentence Transformer model...')
model_name = 'paraphrase-multilingual-mpnet-base-v2' 
model = SentenceTransformer(model_name)

# Encode sellers' combined_text
logging.info('Encoding sellers\' text...')
sellers_df['embedding'] = sellers_df['combined_text'].apply(lambda x: model.encode(x, convert_to_tensor=True))

# Prepare seller coordinates for BallTree (in radians)
seller_coords = np.radians(sellers_df[['latitude', 'longitude']].values)
ball_tree = BallTree(seller_coords, metric='haversine')

# Set similarity threshold
similarity_threshold = 0.75

# Initialize matches list
matches = []

# Earth's radius in kilometers
earth_radius = 6371.0
radius_km = 50.0
radius_rad = radius_km / earth_radius  # Convert km to radians

logging.info('Starting matching process...')

for i, buyer_row in buyers_df.iterrows():
    buyer_coord = np.radians([buyer_row['latitude'], buyer_row['longitude']]).reshape(1, -1)
    # Query BallTree for sellers within 50 km
    indices = ball_tree.query_radius(buyer_coord, r=radius_rad)[0]
    
    if len(indices) == 0:
        continue  # No sellers within 50 km
    
    # Encode buyer's combined_text
    buyer_embedding = model.encode(buyer_row['combined_text'], convert_to_tensor=True).cpu().numpy().reshape(1, -1)
    
    # Extract seller embeddings
    sellers_embeddings_subset = np.vstack(sellers_df.iloc[indices]['embedding'].apply(lambda x: x.cpu().numpy()))
    
    # Compute cosine similarity
    similarity_scores = cosine_similarity(buyer_embedding, sellers_embeddings_subset)[0]
    
    # Find indices where similarity is above threshold
    matching_sellers_indices = np.where(similarity_scores >= similarity_threshold)[0]
    
    for idx in matching_sellers_indices:
        seller_idx = indices[idx]
        seller_row = sellers_df.iloc[seller_idx]
        similarity = similarity_scores[idx]
        # Store the match
        match = {
            'buyer_title': buyer_row['title'],
            'buyer_summary': buyer_row['description'],
            'buyer_long_description': buyer_row['long_description'],
            'buyer_location': buyer_row['state_city'],
            'buyer_latitude': buyer_row['latitude'],
            'buyer_longitude': buyer_row['longitude'],
            'seller_title': seller_row['title'],
            'seller_summary': seller_row['description'],
            'seller_long_description': seller_row['long_description'],
            'seller_location': seller_row['state_city'],
            'seller_latitude': seller_row['latitude'],
            'seller_longitude': seller_row['longitude'],
            'buyer_url': buyer_row['url'],
            'seller_url': seller_row['url'],
            'similarity': similarity,
            'distance_km': geodesic(
                (buyer_row['latitude'], buyer_row['longitude']),
                (seller_row['latitude'], seller_row['longitude'])
            ).kilometers
        }
        matches.append(match)
    
    if (i + 1) % 100 == 0:
        logging.info(f"Processed {i + 1}/{len(buyers_df)} buyers.")

# Create DataFrame from matches
matches_df = pd.DataFrame(matches)

logging.info(f"Total matches found: {len(matches_df)}")

# Optionally, save the matches to a CSV file
matches_df.to_csv('./data/matched_buyers_sellers.csv', index=False)
logging.info("Matching process completed and results saved to 'matched_buyers_sellers.csv'.")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abbasm1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:root:Loading buyer and seller datasets...
INFO:root:Preprocessing buyers' text fields...
INFO:root:Preprocessing sellers' text fields...
INFO:root:Combining text fields...
INFO:root:Extracting state and city from location...
INFO:root:Geocoding buyer locations...
INFO:root:Geocoding seller locations...
INFO:root:Dropped 359 buyers due to failed geocoding.
INFO:root:Dropped 1348 sellers due to failed geocoding.
INFO:root:Loading the Sentence Transformer model...
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
INFO:root:Encoding sellers' text...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:Processed 400/401 buyers.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:Total matches found: 965
INFO:root:Matching process completed and results saved to 'matched_buyers_sellers.csv'.
